In [1]:
import pandas as pd
import geopandas as gpd
import json
import requests
import numpy as np
from shapely.geometry import Point

In [2]:
def traveltimeTransit(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-01-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
    r = requests.get(request)
    data=r.json()
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [3]:
def traveltimeDrive(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,CAR&date=05-01-2021&time="\
    +time+"&maxWalkDistance=750"
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [4]:
def traveltimeWalk(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK&date=05-01-2021&time="\
    +time+"&maxWalkDistance=50000"
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [5]:
def get_stop_url(geom):
    # print(df)
    # geom = df.geometry
    initial='http://127.0.0.1:5000/nearest/v1/driving/'
    end='?number=1'
    point = str(geom.x)+","+str(geom.y)
    requests2 = []
    requests2.append(initial+point+end)
    return requests2
    

In [6]:
import time
def nearby_points(reqs):
    url=reqs[0]
    #to remove any connection timeouts and port usage exceptions
    # r=None
    # l=None
    try:
        r=requests.get(url)
        data=r.json()
        l=Point(data['waypoints'][0]['location'])
        return l
    except:
        time.sleep(60)
        
    # return l

In [7]:
lodes = pd.read_csv('hamilton_all_geographies_od_main_JT00_2018.csv', index_col=[0])
tracts  = gpd.read_file('../Tennessee Census Tracts/tl_2016_47_tract.shp')

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [8]:
lodes_hamilton = lodes[lodes['home_county'] == "Hamilton County, TN"]
lodes_hamilton = lodes_hamilton[lodes_hamilton['work_county'] == "Hamilton County, TN"]
lodes_hamilton['home_tract'] = lodes_hamilton['home_tract'].astype("string")
lodes_hamilton['work_tract'] = lodes_hamilton['work_tract'].astype("string")
lodes_hamilton['home_geocode'] = lodes_hamilton['home_geocode'].astype("string")
lodes_hamilton['work_geocode'] = lodes_hamilton['work_geocode'].astype("string")

In [9]:
tracts_hamilton = tracts[tracts['COUNTYFP'] == "065"]

In [10]:
# home_merged = pd.merge(lodes_hamilton, tracts_hamilton, left_on=['home_tract'], right_on=['GEOID'], how='left')

In [11]:
hw_tracts = lodes_hamilton.groupby(['home_tract', 'work_tract']).sum().sort_values('total_jobs', ascending=False).reset_index()[['home_tract', 'work_tract', 'total_jobs']].dropna()

In [12]:
hw_tracts_geom = hw_tracts.merge(tracts_hamilton[['GEOID', 'geometry']], left_on=['home_tract'], right_on=['GEOID'], how='left')
hw_tracts_geom = hw_tracts_geom.rename({'geometry': 'home_geom' }, axis='columns')

hw_tracts_geom = hw_tracts_geom.merge(tracts_hamilton[['GEOID', 'geometry']], left_on=['work_tract'], right_on=['GEOID'], how='left')
hw_tracts_geom = hw_tracts_geom.rename({'geometry': 'work_geom' }, axis='columns')
hw_tracts_geom.drop(['GEOID_x', 'GEOID_y'], axis=1, inplace=True)

In [13]:
hw_tracts_geom['home_geom_centroid'] = hw_tracts_geom.home_geom.apply(lambda x : x.centroid)
hw_tracts_geom['work_geom_centroid'] = hw_tracts_geom.work_geom.apply(lambda x : x.centroid)

In [12]:
# hw_tracts_geom.iloc[0].home_geom_centroid_x
# traveltimeDrive(hw_tracts_geom.iloc[0].home_geom_centroid_y, hw_tracts_geom.iloc[0].home_geom_centroid_x, hw_tracts_geom.iloc[0].work_geom_centroid_y, hw_tracts_geom.iloc[0].work_geom_centroid_x, '10:00am')
# requests.get('http://localhost:8080/otp/routers/default/plan?fromPlace=35.06091486239899%2C-85.26351928710938&toPlace=34.969249651079004%2C-85.29853820800781&time=10%3A00am&date=05-01-2022&mode=TRANSIT%2CWALK&maxWalkDistance=4828.032&arriveBy=false&wheelchair=false&showIntermediateStops=true&debugItineraryFilter=false&locale=en')

In [14]:
hw_tracts_geom['hc_url'] = hw_tracts_geom.home_geom_centroid.apply(get_stop_url)
hw_tracts_geom['wc_url'] = hw_tracts_geom.work_geom_centroid.apply(get_stop_url)

In [15]:
hw_tracts_geom['hc_nearby']=hw_tracts_geom.hc_url.apply(nearby_points)
hw_tracts_geom['wc_nearby']=hw_tracts_geom.wc_url.apply(nearby_points)

In [ ]:
hw_tracts_geom['hc_nearby_x'] = hw_tracts_geom.hc_nearby.apply(lambda pt : pt.x)
hw_tracts_geom['hc_nearby_y'] = hw_tracts_geom.hc_nearby.apply(lambda pt : pt.y)

hw_tracts_geom['wc_nearby_x'] = hw_tracts_geom.wc_nearby.apply(lambda pt : pt.x)
hw_tracts_geom['wc_nearby_y'] = hw_tracts_geom.wc_nearby.apply(lambda pt : pt.y)

# hw_tracts_geom['hc_nearby_x'] = hw_tracts_geom.home_geom_centroid.apply(lambda pt : pt.x)
# hw_tracts_geom['hc_nearby_y'] = hw_tracts_geom.home_geom_centroid.apply(lambda pt : pt.y)

# hw_tracts_geom['wc_nearby_x'] = hw_tracts_geom.work_geom_centroid.apply(lambda pt : pt.x)
# hw_tracts_geom['wc_nearby_y'] = hw_tracts_geom.work_geom_centroid.apply(lambda pt : pt.y)

In [ ]:
for i in range(hw_tracts_geom.shape[0]):
    hw_tracts_geom.loc[i, 'transit_time_fn'] = traveltimeTransit(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'drive_time_fn'] = traveltimeDrive(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'walk_time_fn'] = traveltimeWalk(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'ratio'] = np.divide(hw_tracts_geom.loc[i, 'drive_time_fn'], hw_tracts_geom.loc[i, 'transit_time_fn'])

In [ ]:
hw_tracts_geom.groupby(['home_tract']).mean().merge(tracts_hamilton[['GEOID', 'geometry']], left_on='home_tract', right_on='GEOID').reset_index()[['GEOID', 'ratio', 'geometry']].to_csv('home_tracts_transit_mean_5km_2.csv', index=False)

In [73]:
# hw_tracts_geom.drop(['home_geom_centroid_x', 'home_geom_centroid_y', 'work_geom_centroid_x', 'work_geom_centroid_y'], axis=1)
# hw_tracts_geom.groupby(['home_tract']).mean().reset_index()[['home_tract', 'ratio', 'geometry']].to_csv('home_tracts_transit_mean_5km_2.csv', index=False)

In [21]:
# hw_tracts_geom[hw_tracts_geom.ratio.notnull()].home_tract.unique()
# hw_tracts_geom[hw_tracts_geom.ratio.notnull()].head(1)
# [['home_tract', 'work_tract', 'total_jobs', 'drive_time', 'transit_time', 'ratio']]

In [ ]:
blocks  = gpd.read_file('../Tennessee Census Block/tl_2020_47_tabblock20.shp')

In [23]:
blocks.head(1)

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,47,085,130100,3014,470851301003014,Block 3014,G5040,None,None,None,S,16551,0,+36.1064600,-087.6384045,"POLYGON ((-87.63927 36.10687, -87.63902 36.106..."


In [ ]:
blocks_hamilton = blocks[blocks['COUNTYFP20'] == "065"]

In [25]:
len(blocks_hamilton.GEOID20.unique())

7076

In [26]:
hw_blocks = lodes_hamilton.groupby(['home_geocode', 'work_geocode']).sum().sort_values('total_jobs', ascending=False).reset_index()[['home_geocode', 'work_geocode', 'total_jobs']].dropna()

In [26]:
# lodes[lodes['home_county'] == "Davidson County, TN"]
# blocks_hamilton
# hw_blocks.merge(blocks_hamilton[['GEOID20', 'geometry']], left_on=['home_geocode'], right_on=['GEOID20'], how='left')

In [27]:
blocks[blocks['GEOID20'] == '470659802001160']

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry


In [28]:
# lodes_hamilton.groupby(['home_geocode', 'work_geocode']).sum().sort_values('total_jobs', ascending=False).reset_index().dropna()

In [28]:
hw_blocks_geom = hw_blocks.merge(blocks_hamilton[['GEOID20', 'geometry']], left_on=['home_geocode'], right_on=['GEOID20'], how='left')
hw_blocks_geom = hw_blocks_geom.rename({'geometry': 'home_geom' }, axis='columns')

hw_blocks_geom = hw_blocks_geom.merge(blocks_hamilton[['GEOID20', 'geometry']], left_on=['work_geocode'], right_on=['GEOID20'], how='left')
hw_blocks_geom = hw_blocks_geom.rename({'geometry': 'work_geom' }, axis='columns')
hw_blocks_geom.drop(['GEOID20_x', 'GEOID20_y'], axis=1, inplace=True)

In [29]:
hw_blocks_geom_drp =  hw_blocks_geom.dropna().reset_index(drop=True)

In [30]:
hw_blocks_geom_drp['home_geom_centroid'] = hw_blocks_geom_drp.home_geom.apply(lambda x : x.centroid)
hw_blocks_geom_drp['work_geom_centroid'] = hw_blocks_geom_drp.work_geom.apply(lambda x : x.centroid)

In [32]:
# hw_tracts_geom.iloc[0].home_geom_centroid_x
# traveltimeDrive(hw_tracts_geom.iloc[0].home_geom_centroid_y, hw_tracts_geom.iloc[0].home_geom_centroid_x, hw_tracts_geom.iloc[0].work_geom_centroid_y, hw_tracts_geom.iloc[0].work_geom_centroid_x, '10:00am')
# requests.get('http://localhost:8080/otp/routers/default/plan?fromPlace=35.06091486239899%2C-85.26351928710938&toPlace=34.969249651079004%2C-85.29853820800781&time=10%3A00am&date=05-01-2022&mode=TRANSIT%2CWALK&maxWalkDistance=4828.032&arriveBy=false&wheelchair=false&showIntermediateStops=true&debugItineraryFilter=false&locale=en')

In [38]:
hw_blocks_geom_drp['hc_url'] = hw_blocks_geom_drp.home_geom_centroid.apply(get_stop_url)
hw_blocks_geom_drp['wc_url'] = hw_blocks_geom_drp.work_geom_centroid.apply(get_stop_url)

In [55]:
hw_blocks_geom_drp['hc_nearby']=hw_blocks_geom_drp.hc_url.apply(nearby_points)
hw_blocks_geom_drp['wc_nearby']=hw_blocks_geom_drp.wc_url.apply(nearby_points)

In [63]:
hw_blocks_geom_drp = hw_blocks_geom_drp.dropna(subset=['hc_nearby', 'wc_nearby'])

In [67]:
hw_blocks_geom_drp['hc_nearby_x'] = hw_blocks_geom_drp.hc_nearby.apply(lambda pt : pt.x)
hw_blocks_geom_drp['hc_nearby_y'] = hw_blocks_geom_drp.hc_nearby.apply(lambda pt : pt.y)

hw_blocks_geom_drp['wc_nearby_x'] = hw_blocks_geom_drp.wc_nearby.apply(lambda pt : pt.x)
hw_blocks_geom_drp['wc_nearby_y'] = hw_blocks_geom_drp.wc_nearby.apply(lambda pt : pt.y)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [71]:
hw_blocks_geom_drp.shape[0]

36625

In [72]:
for i in range(hw_blocks_geom_drp.shape[0]):
    hw_blocks_geom_drp.loc[i, 'transit_time_fn'] = traveltimeTransit(hw_blocks_geom_drp.iloc[i].hc_nearby_y, hw_blocks_geom_drp.iloc[i].hc_nearby_x, hw_blocks_geom_drp.iloc[i].wc_nearby_y, hw_blocks_geom_drp.iloc[i].wc_nearby_x, '10:00am')
    hw_blocks_geom_drp.loc[i, 'drive_time_fn'] = traveltimeDrive(hw_blocks_geom_drp.iloc[i].hc_nearby_y, hw_blocks_geom_drp.iloc[i].hc_nearby_x, hw_blocks_geom_drp.iloc[i].wc_nearby_y, hw_blocks_geom_drp.iloc[i].wc_nearby_x, '10:00am')
    # hw_tracts_geom.loc[i, 'walk_time_fn'] = traveltimeWalk(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_blocks_geom_drp.loc[i, 'ratio'] = np.divide(hw_blocks_geom_drp.loc[i, 'drive_time_fn'], hw_blocks_geom_drp.loc[i, 'transit_time_fn'])

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
hw_blocks_geom_drp.groupby('home_geocode')['ratio'].mean().reset_index().to_csv('blocks_transit_ratios.csv', index=False)

In [74]:
hw_blocks_geom_drp
# .groupby('home_geocode')['ratio', 'geometry'].mean().dropna().reset_index()

,home_geocode,work_geocode,total_jobs,home_geom,work_geom,home_geom_centroid,work_geom_centroid,hc_url,wc_url,hc_nearby,wc_nearby,hc_nearby_x,hc_nearby_y,wc_nearby_x,wc_nearby_y,transit_time_fn,drive_time_fn,ratio
0,470650114474000,470650112032000,41.0,"POLYGON ((-85.10962 35.14735, -85.10924 35.148...","POLYGON ((-85.04847 35.05330, -85.04801 35.053...",POINT (-85.08853947589893 35.15051021884101),POINT (-85.04319800662712 35.05058522551829),[http://127.0.0.1:5000/nearest/v1/driving/-85....,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.085202 35.148474),POINT (-85.043999 35.050091),-85.085202,35.148474,-85.043999,35.050091,NaN,2031.0,NaN
1,470650113214000,470650004001044,38.0,"POLYGON ((-85.14462 35.02718, -85.14211 35.031...","POLYGON ((-85.26994 35.04209, -85.26987 35.042...",POINT (-85.13907434746974 35.03032259403921),POINT (-85.26914889397058 35.042138587216485),[http://127.0.0.1:5000/nearest/v1/driving/-85....,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.139669 35.030262),POINT (-85.269313 35.04185),-85.139669,35.030262,-85.269313,35.041850,4893.0,1689.0,0.345187
2,470650114474000,470650004001044,37.0,"POLYGON ((-85.10962 35.14735, -85.10924 35.148...","POLYGON ((-85.26994 35.04209, -85.26987 35.042...",POINT (-85.08853947589893 35.15051021884101),POINT (-85.26914889397058 35.042138587216485),[http://127.0.0.1:5000/nearest/v1/driving/-85....,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.085202 35.148474),POINT (-85.269313 35.04185),-85.085202,35.148474,-85.269313,35.041850,NaN,3010.0,NaN
3,470650112031017,470650112032000,35.0,"POLYGON ((-85.08294 35.02482, -85.08266 35.025...","POLYGON ((-85.04847 35.05330, -85.04801 35.053...",POINT (-85.08137676068219 35.025033544726156),POINT (-85.04319800662712 35.05058522551829),[http://127.0.0.1:5000/nearest/v1/driving/-85....,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.080987 35.024533),POINT (-85.043999 35.050091),-85.080987,35.024533,-85.043999,35.050091,NaN,762.0,NaN
4,470650113142000,470650112032000,30.0,"POLYGON ((-85.09636 35.03232, -85.09644 35.032...","POLYGON ((-85.04847 35.05330, -85.04801 35.053...",POINT (-85.08993103287136 35.034329558920675),POINT (-85.04319800662712 35.05058522551829),[http://127.0.0.1:5000/nearest/v1/driving/-85....,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.089688 35.034746),POINT (-85.043999 35.050091),-85.089688,35.034746,-85.043999,35.050091,NaN,743.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36628,470650104112000,470650007003010,1.0,"POLYGON ((-85.20524 35.18946, -85.20497 35.190...","POLYGON ((-85.29477 35.07082, -85.29467 35.070...",POINT (-85.20441906687675 35.189756164803306),POINT (-85.29188365934857 35.07058021237548),[http://127.0.0.1:5000/nearest/v1/driving/-85....,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.204081 35.189662),POINT (-85.291943 35.070513),-85.204081,35.189662,-85.291943,35.070513,NaN,NaN,NaN
10530,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4320.0,970.0,0.224537
15687,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1126.0,NaN
26295,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3942.0,838.0,0.212582


# new block data

In [102]:
bl_ham_gdf = gpd.read_file('HamiltonCensusBlocks.geojson')

In [103]:
hw_blocks_geom2 = hw_blocks.merge(bl_ham_gdf[['geoid20', 'geometry']], left_on=['home_geocode'], right_on=['geoid20'], how='left')
hw_blocks_geom2 = hw_blocks_geom2.rename({'geometry': 'home_geom' }, axis='columns')

hw_blocks_geom2 = hw_blocks_geom2.merge(bl_ham_gdf[['geoid20', 'geometry']], left_on=['work_geocode'], right_on=['geoid20'], how='left')
hw_blocks_geom2 = hw_blocks_geom2.rename({'geometry': 'work_geom' }, axis='columns')
hw_blocks_geom2.drop(['geoid20_x', 'geoid20_y'], axis=1, inplace=True)

In [121]:
hw_blocks_geom2[['work_geocode', 'total_jobs']].merge(bl_ham_gdf[['geoid20', 'geometry']], left_on=['work_geocode'], right_on=['geoid20'], how='right').sort_values('total_jobs', ascending=False)

,work_geocode,total_jobs,geoid20,geometry
36705,470650112032000,41.0,470650112032000,"POLYGON ((-85.04847 35.05330, -85.04800 35.053..."
42193,470650004001044,38.0,470650004001044,"POLYGON ((-85.26994 35.04209, -85.26987 35.042..."
42194,470650004001044,37.0,470650004001044,"POLYGON ((-85.26994 35.04209, -85.26987 35.042..."
42195,470650004001044,36.0,470650004001044,"POLYGON ((-85.26994 35.04209, -85.26987 35.042..."
36706,470650112032000,35.0,470650112032000,"POLYGON ((-85.04847 35.05330, -85.04800 35.053..."
...,...,...,...,...
57192,NaN,NaN,470650018002052,"POLYGON ((-85.33110 34.98983, -85.33109 34.990..."
57193,NaN,NaN,470650018002059,"POLYGON ((-85.33242 34.98580, -85.33208 34.986..."
57194,NaN,NaN,470650018002048,"POLYGON ((-85.32838 34.99160, -85.32834 34.992..."
57195,NaN,NaN,470650018002033,"POLYGON ((-85.32728 34.99074, -85.32711 34.991..."


In [109]:
# lodes_hamilton[lodes_hamilton['work_county'] == "Hamilton County, TN"]

In [122]:
bl_ham = gpd.read_file('BlockPopulations.geojson')
bl_ham_g = bl_ham[['geoid', 'geometry']]

In [129]:
hw_blocks_geom2[['work_geocode', 'total_jobs']].merge(bl_ham_g[['geoid', 'geometry']], left_on=['work_geocode'], right_on=['geoid'], how='left').sort_values('total_jobs', ascending=False)

,work_geocode,total_jobs,geoid,geometry
0,470659802001160,66,NaN,None
1,470650112032000,41,470650112032000,"POLYGON ((-85.04847 35.05330, -85.04800 35.053..."
2,470650004001044,38,470650004001044,"POLYGON ((-85.26994 35.04209, -85.26987 35.042..."
3,470650004001044,37,470650004001044,"POLYGON ((-85.26994 35.04209, -85.26987 35.042..."
4,470650004001044,36,470650004001044,"POLYGON ((-85.26994 35.04209, -85.26987 35.042..."
...,...,...,...,...
38456,470650031001025,1,470650031001025,"POLYGON ((-85.31152 35.04390, -85.31129 35.043..."
38455,470650031001036,1,470650031001036,"POLYGON ((-85.30957 35.03847, -85.30955 35.038..."
38454,470650031001050,1,NaN,None
38453,470650031001056,1,NaN,None


In [131]:
blocks = blocks[['GEOID20', 'geometry']]

In [142]:
# tracts_hamilton
import itertools as it
permutations = list(it.permutations(tracts_hamilton['GEOID']))

MemoryError: 